# Movie Review ML

## Importações 

### Modelos

Modelos de classificação linear, Naive Bayes e SVN como soluções mais utilizadas nas pesquisas que são simples e que atingem o objetivo de lidar com textos.

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

### Engenharia de características

Foram utilizadas as bibliotecas TextBlob e Tfidf, sendo essa pŕopria da Scikit learn 

In [ ]:
from dense_transform import DenseTransformer, tokenizer
from sklearn.feature_extraction.text import TfidfTransformer,  CountVectorizer, TfidfVectorizer

## Gráficos

Para a geração de gráficos foi utilizado o Matplot lib

In [ ]:
from matplotlib import pyplot